In [224]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [225]:
df = pd.read_csv('all_stocks_5yr.csv').dropna()


In [226]:
companies = df.groupby("Name")


In [227]:
block_size = 50

In [228]:
train_prices = []
test_prices = []
features = ['open', 'volume', 'high', 'low', 'close']
for name, company_df in companies:
    for i in range(0, len(company_df) - block_size + 1, block_size):
        block = company_df.iloc[i:i+block_size][features]
        train_prices.append(block.values.tolist())
    test_prices.append(train_prices.pop())
train_prices = torch.tensor(train_prices)
test_prices = torch.tensor(test_prices)

# Reshape to 2D for scaling
train_prices_flat = train_prices.view(-1, len(features))
test_prices_flat = test_prices.view(-1, len(features))

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform it
train_prices_scaled_flat = scaler.fit_transform(train_prices_flat)

# Transform the test data using the same scaler
test_prices_scaled_flat = scaler.transform(test_prices_flat)

# Reshape back to the original shape
train_prices_scaled = torch.tensor(train_prices_scaled_flat, dtype=torch.float32).view(train_prices.shape)
test_prices_scaled = torch.tensor(test_prices_scaled_flat, dtype=torch.float32).view(test_prices.shape)

print(train_prices_scaled.shape, test_prices_scaled.shape)

torch.Size([11776, 50, 5]) torch.Size([505, 50, 5])


In [250]:
from torch import nn
import torch
import math
from torch.nn import functional as F
from torchtune.modules import RMSNorm, RotaryPositionalEmbeddings


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        scaled_hidden = int(2 / 3 * 4 * config.emb_dim)
        self.fc1 = nn.Linear(config.emb_dim, scaled_hidden, bias=False)
        self.fc2 = nn.Linear(config.emb_dim, scaled_hidden, bias=False)
        self.fc3 = nn.Linear(scaled_hidden, config.emb_dim, bias=False)

    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        hidden = F.silu(x1)
        hidden = hidden * x2
        return self.fc3(hidden)


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.emb_dim % config.n_head == 0
        self.emb_dim = config.emb_dim
        self.n_head = config.n_head
        self.head_dim = config.emb_dim // config.n_head

        self.Wq = nn.Linear(config.emb_dim, self.n_head * self.head_dim, bias=False)
        self.Wk = nn.Linear(config.emb_dim, self.n_head * self.head_dim, bias=False)
        self.Wv = nn.Linear(config.emb_dim, self.n_head * self.head_dim, bias=False)
        self.Wo = nn.Linear(config.emb_dim, self.n_head * self.head_dim, bias=False)
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size)).view(
                1, 1, config.block_size, config.block_size
            ),
        )
        self.pos_emb = RotaryPositionalEmbeddings(self.head_dim, config.block_size)

    def forward(self, x):
        batch_size, seq_len, dim = x.shape
        assert dim == self.emb_dim, "dim must be equal to self.emb_dim"

        xq = self.Wq(x)
        xk = self.Wk(x)
        xv = self.Wv(x)

        xq = xq.view(batch_size, seq_len, self.n_head, self.head_dim)
        xk = xk.view(batch_size, seq_len, self.n_head, self.head_dim)
        xv = xv.view(batch_size, seq_len, self.n_head, self.head_dim)

        xq = self.pos_emb(xq)
        xk = self.pos_emb(xk)

        xq = xq.transpose(1, 2)
        keys = xk.transpose(1, 2)
        values = xv.transpose(1, 2)

        scores = torch.matmul(xq, keys.transpose(-2, -1)) / math.sqrt(self.head_dim)
        scores = scores.masked_fill(
            self.bias[:, :, :seq_len, :seq_len] == 0, float("-inf")
        )
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        context = torch.matmul(scores, values)

        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        output = self.Wo(context)
        return output


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.rn1 = RMSNorm(config.emb_dim)
        self.rn2 = RMSNorm(config.emb_dim)
        self.attn = MultiHeadSelfAttention(config)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.rn1(x))
        x = x + self.mlp(self.rn2(x))
        return x


class LLama(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # self.inp_emb = nn.Embedding(config.vocab_size, config.emb_dim)
        self.inp_emb = nn.Linear(config.num_features, config.emb_dim)
        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layers)])
        self.fc_out = nn.Linear(config.emb_dim, 1, bias=False)
        self.rmsnorm = RMSNorm(config.emb_dim)
        # self.inp_emb.weight = self.fc_out.weight # https://paperswithcode.com/method/weight-tying

    def forward(self, x, y=None):
        batch, seq_len, features = x.shape
        x = self.inp_emb(x)
        for block in self.blocks:
            x = block(x)
        x = self.rmsnorm(x)

        logits = self.fc_out(x)
        loss = None
        # print(logits.view(-1).squeeze().shape, y.view(-1).shape)
        logits = logits.squeeze()
        print(logits.shape, y.shape)
        if y is not None:
            loss = F.mse_loss(
                logits.view(-1).squeeze(), y.view(-1), reduction='mean'
            )
        return logits, loss

    @torch.no_grad()
    def generate(self, inp, temperature=1.0, top_k=None):
        inp = inp.reshape(1, -1)
        for _ in range(self.config.block_size - inp.shape[1]):
            logits, _ = self.forward(inp)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("Inf")
            probs = F.softmax(logits, dim=-1)
            inp_next = torch.multinomial(probs, num_samples=1)
            inp = torch.cat((inp, inp_next), dim=1)
        return inp[0]

In [251]:
class Config:
    def __init__(
        self, num_features, emb_dim, n_layers, n_head, block_size, top_k=None):
        self.block_size = block_size
        self.window_size = self.block_size // 2
        self.batch_size = 32
        self.iters = 1000
        self.dropout = 0.1
        self.n_kv_heads = 8
        self.num_features = num_features
        self.top_k = top_k
        self.emb_dim = emb_dim
        self.n_layers = n_layers
        self.n_head = n_head


In [252]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

config = Config(len(features), 512, 8, 8, block_size)
model = LLama(config)
model.to(device)
optim = torch.optim.AdamW(model.parameters(), lr=3e-5)
iters = 100
batch_size = 32
train_losses = []
test_losses = []

num_batches = len(train_prices_scaled) // batch_size

for i in range(iters):
    batch_index = i % num_batches
    
    start_idx = batch_index * batch_size
    end_idx = (batch_index + 1) * batch_size
    x_train = train_prices_scaled[start_idx:end_idx]
    
    optim.zero_grad()
    
    logits, train_loss = model(x_train[:, :-1, :], x_train[:, 1:, 0])
    
    train_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optim.step()
    
    train_losses.append(train_loss.item())
    
    if i % 10 == 0:
        print(i)
        with torch.no_grad():
            x_test = test_prices_scaled
            _, test_loss = model(x_test[:, :-1, :], x_test[:, 1:, 0])
            test_losses.append((i, test_loss.item()))
        print(train_loss.item(), test_loss.item())


test_loss_indices, test_loss_values = zip(*test_losses)

plt.figure(figsize=(10, 6))
plt.plot(range(iters), train_losses, label='Training Loss')
plt.plot(test_loss_indices, test_loss_values, label='Testing Loss', marker='o')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training and Testing Loss over Iterations')
plt.legend()
plt.grid(True)
plt.show()


torch.Size([32, 49]) torch.Size([32, 49])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.